<a href="https://colab.research.google.com/github/yarijy/pt/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# TODO BEFORE YOU START: Switch to GPU instance 
# via menu Edit->Notebook torch.cuda.device_count()settings->Acceleration->GPU->Save

In [0]:
!pip install torch > /dev/null

In [0]:
import torch
assert torch.cuda.is_available() == True
assert torch.cuda.device_count() >= 1

In [0]:
"""
Hey, Transformer, talk to me!
Practical text generation with large-scale neural networks.
Thanks to Data Science UA
"""

"""
Re-run this cell if your token is expired
"""
import random
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open_by_key('1UEgjficfHAgOxfaZFDoBnQmEECJ8jfbw9IUnnaVR0h0')

In [0]:
# TODO: Change to your personal ID
user_id = 20

Simple stupid text generator to test public board

In [9]:
# Testing public results board

# Random words list:
words = '''word,but,world,some,we,out,hello,other,were,all,go,there,all,up,use,your,hello,
said,I,she,which,do,there,time,hello,will,way,about,ready,then,hello,write,ready,
like,hello,to,ready,long,make,me,hello,see,two,has,look,more,day'''

random_text = random.choices(words.replace('\n',',').strip().split(','), k=4)
text = ' '.join(random_text)

# Publish results
worksheet.sheet1.update_cell(user_id, 2, f"default: {text}")
print(text)

# TODO: modify or create simplest text generator
# worksheet.sheet1.update_cell(user_id, 2, f"custom: {text}")

see way world up


Markov chain text generator

In [0]:
!pip install markovify goose3 > /dev/null

Import content


In [0]:
# https://github.com/goose3/goose3
from goose3 import Goose
g = Goose()

# Replace with your own URL (English language pls)
url = 'https://www.breakingtravelnews.com/news/article/austrian-airlines-secures-600m-in-bailout-funds/'

def crawl_content(url: str) -> str:
    article = g.extract(url=url)
    text = article.cleaned_text.strip()
    return text

In [17]:
text = crawl_content(url=url)
length = len(text)
worksheet.sheet1.update_cell(user_id, 3, f"len: {length} | content: {text[:50]}...")
print(text[:250])

Austrian Airlines has secured a €600 million rescue package from the national government and owner Lufthansa Group.

The carrier said the financial assistance was designed to support efforts to sustainably safeguard Vienna as an aviation hub in the l


In [18]:
# https://github.com/jsvine/markovify
import markovify
text = text.replace("\n"," ")

# Build the model
simple_model = markovify.Text(text)

# Print generated sentence
simple_sentence = simple_model.make_sentence()
worksheet.sheet1.update_cell(user_id, 4, f"markov: {simple_sentence}")
print(simple_sentence)

The Austrian government issued €150 million in an injection of equity capital and the consent of the EU Commission.


In [20]:
# TODO combine a few chains, modify on your choice

text_a = crawl_content('https://www.breakingtravelnews.com/news/article/austrian-airlines-secures-600m-in-bailout-funds/')
text_b = crawl_content('http://news.mit.edu/2020/student-geeticka-chauhan-0609')
text_c = crawl_content('https://en.wikipedia.org/wiki/Donald_Trump')

simple_model_a = markovify.Text(text_a)
simple_model_b = markovify.Text(text_b)
simple_model_c = markovify.Text(text_c)

combined_model = markovify.combine([ simple_model_a, simple_model_b, simple_model_c], [ 1.2, 1, 1.2])
combined_sentence = combined_model.make_sentence()
worksheet.sheet1.update_cell(user_id, 5, f"markov comb: {combined_sentence}")
print(combined_sentence)

According to the Austrian Federal Government has received long-term, binding commitments about the development of the derelict Commodore Hotel, adjacent to Grand Central Terminal.


In [0]:
!pip install transformers torch > /dev/null

In [22]:
#Validator
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

ac_tokenizer = AutoTokenizer.from_pretrained("textattack/distilbert-base-cased-CoLA")
ac_model = AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-cased-CoLA")
ac_model = ac_model.to("cuda")

def acceptibility_check(text: str) -> float:
    seq_a, seq_b = text[:int(len(text)/2)], text[int(len(text)/2):]
    classes = ["not acceptable", "acceptable"]
    acceptable = ac_tokenizer.encode_plus(seq_a, seq_b, return_tensors="pt")
    acceptable.to("cuda")
    acceptable_classification_logits = ac_model(**acceptable)[0]
    acceptable_classification_logits.to("cuda")
    acceptable_results = torch.softmax(acceptable_classification_logits, dim=1).tolist()[0]
    return acceptable_results[1]

#Some tests to check ligustic acceptibility validation
assert acceptibility_check("This is a simple sentence for ligustic acceptibility validation") > 0.8
assert acceptibility_check("Simple is a simple sentence for up not bad test") < 0.4

print("acceptibility_check is functional: ok")


acceptibility_check is functional: ok


In [23]:
# Sentiment analysis
from transformers import pipeline
sentiment_analysis = pipeline('sentiment-analysis')

assert sentiment_analysis("This workshop is awesome!")[0]['label'] == "POSITIVE"
assert sentiment_analysis("It's bad that we don't have 100 participants today!")[0]['label'] == "NEGATIVE"
print("sentiment_analysis is functional: ok")


sentiment_analysis is functional: ok


Let's check our text

In [26]:
# Check our text
acceptibility = acceptibility_check(combined_sentence)
sentiment = sentiment_analysis(combined_sentence)[0]

worksheet.sheet1.update_cell(user_id, 6, f"markov acceptibility: {acceptibility} | sentiment: {sentiment}")
print(f"markov acceptibility: {acceptibility} | sentiment: {sentiment}")

markov acceptibility: 0.8621480464935303 | sentiment: {'label': 'POSITIVE', 'score': 0.8367653489112854}


In [27]:
# Post the result
worksheet.sheet1.update_cell(user_id, 7, simple_sentence)

{'spreadsheetId': '1UEgjficfHAgOxfaZFDoBnQmEECJ8jfbw9IUnnaVR0h0',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': "'t1'!G20",
 'updatedRows': 1}

In [0]:
#Transformers

In [28]:
#GPT2
from transformers import pipeline
text_generator = pipeline("text-generation")

In [29]:
generated = text_generator("hello world", max_length=85)
print(generated)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': 'hello world champions in Rio last year was born at the same time the British team was formed.\n\nThe two teams did not meet in a regular season finale, but they were in every way connected by mutual interest. That is because Australia\'s only goal was to win the World Cup of Hockey.\n\n"Everyone knows that and has been a part of the team this entire year," said Kallo, as he'}]


In [31]:
def line_validator(text: str, max_len: int = 20) -> bool:
    return True if 3 < len(text) < max_len else False

def output_list_validator(list_of_text:list) -> list:
    output_list = []
    for line in list_of_text:
        if line not in output_list and line_validator(line):
            output_list.append(line)

    return output_list
    
def random_list_sample(sample:list) -> list:
    return random.choices(shop_list, k=3)

def enrich_list(enr_list:list, target_count:int=6) -> list:

    while len(enr_list) < target_count:
        generated = text_generator('\n'.join(random_list_sample(enr_list)), max_length=85)[0]['generated_text']
        enr_list += output_list_validator(generated.split("\n"))
        enr_list = list(set(enr_list))
    
    return enr_list[:target_count]


shop_list = ['chair', 'table', 'sofa', 'cupboard']

#Enrich list with generated samples
print(enrich_list(shop_list, 10))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['butterdough cups', 'paper towels', 'chair', '4x4" plywood', 'machete brushes', '4x6 inch sheeting', 'paper', 'table', 'sofa', 'sofas and blankets']


In [32]:
from transformers import AutoModelWithLMHead, AutoTokenizer

xlnet_model = AutoModelWithLMHead.from_pretrained("xlnet-base-cased")
xlnet_model = xlnet_model.to("cuda")
xlnet_tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet to process short samples
# Sample from https://businesstown.com/how-to-write-an-effective-slogan/
PADDING_TEXT = """From her apartment in Sidney-Pacific, where she has stayed put due to travel restrictions in her home country of India, Chauhan is still learning the ropes of her new position. With others, she has been busy preparing to meet the future challenge of safely redensifying the living space of more than 1000 people: how to regulate high-density common areas, handle noise complaints as people spend more time in their rooms, and care for the mental and physical well-being of a community that can only congregate virtually."""


def xlnet_generate(prompt:str, max_length:int = 400) -> str:
    xlnet_inputs = xlnet_tokenizer.encode( PADDING_TEXT+prompt, add_special_tokens=False, return_tensors="pt")
    xlnet_inputs = xlnet_inputs.to("cuda")
    prompt_length = len(xlnet_tokenizer.decode(xlnet_inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
    outputs = xlnet_model.generate(xlnet_inputs, max_length=max_length, do_sample=True, top_p=0.95, top_k=60)
    return xlnet_tokenizer.decode(outputs[0])[prompt_length:]

In [42]:
def dummy_xlnet_conditional_quote_generation(prompt):
    sample_xlnet_output = ''

    while '"' not in sample_xlnet_output:
        sample_xlnet_output = xlnet_generate(f"{prompt}:\"", 200)
        return sample_xlnet_output.split('"')[0]

xlnet_demo_output = dummy_xlnet_conditional_quote_generation("You meet someone and you fall in love and")
print(xlnet_demo_output)
#Real output samples:

#The t-shirt is not supposed to fit.
#If you know how to write, you can do anything you want.

 you see somebody and you fall in love and:


In [48]:
#Post the result
worksheet.sheet1.update_cell(user_id, 8, xlnet_demo_output)

{'spreadsheetId': '1UEgjficfHAgOxfaZFDoBnQmEECJ8jfbw9IUnnaVR0h0',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': "'t1'!H20",
 'updatedRows': 1}

In [49]:
from tqdm.notebook import trange, tqdm

final_results = []
#Mixing results
for item in tqdm(shop_list):
    accept = 0
    print(accept)

    while accept < 0.8:
        output = dummy_xlnet_conditional_quote_generation(f"What is the best short phrase for my t-shirt? As {item} lover I decided to put this")
        accept = acceptibility_check(output)
        sentiment = sentiment_analysis(output)
        print(output, accept, sentiment)

    final_results.append(output)

0
<eop><eod> The answer:  0.8229870796203613 [{'label': 'NEGATIVE', 'score': 0.9268515110015869}]
0
 the table is very much part of my living space. 0.8222324848175049 [{'label': 'POSITIVE', 'score': 0.9992571473121643}]
0
What is the best short phrase for your t-shirt? What is the most short phrase for your t-shirt? How are the most short phrases for your t-shirt? 0.6032059788703918 [{'label': 'NEGATIVE', 'score': 0.9528470039367676}]
I just want to be here. I never wanted to come here. But I want to be here now. And in time I will find a way to get it all done. If this is the time, then I won't give up any work. I don't care how things feel, just that I want to be here right 0.8200044631958008 [{'label': 'POSITIVE', 'score': 0.995358943939209}]
0
... 0.8902391791343689 [{'label': 'POSITIVE', 'score': 0.9585016965866089}]
0
When you think about it it's like a full-sized sofa, and we are so comfortable. 0.9025592803955078 [{'label': 'POSITIVE', 'score': 0.9996767044067383}]
0
 0.471696

In [50]:
final_results

['<eop><eod> The answer: ',
 ' the table is very much part of my living space.',
 "I just want to be here. I never wanted to come here. But I want to be here now. And in time I will find a way to get it all done. If this is the time, then I won't give up any work. I don't care how things feel, just that I want to be here right",
 '...',
 "When you think about it it's like a full-sized sofa, and we are so comfortable.",
 'You need to use what you have stored and stored.']

In [51]:
#Summarization 
from transformers import pipeline
summarizer = pipeline("summarization")
print(summarizer(PADDING_TEXT, max_length=130, min_length=30))

Your max_length is set to 130, but you input_length is only 106. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': 'Chauhan is still learning the ropes of her new position. She has been busy preparing to meet the future challenge of safely redensifying the living space of more than 1000 people.'}]


In [52]:
#Question-answering
from transformers import pipeline
nlp = pipeline("question-answering")
print(nlp(question="Which city tried to use slogan?", context=PADDING_TEXT))


{'score': 0.391566375023336, 'start': 22, 'end': 37, 'answer': 'Sidney-Pacific,'}


In [0]:
#Final competetion
# Use your imagination to create your own unique pipeline with proposed components for T-Shirt phrase generation in 30 minutes.

In [53]:
# Post your best result
your_best_result = ''
worksheet.sheet1.update_cell(user_id, 9, your_best_result)

{'spreadsheetId': '1UEgjficfHAgOxfaZFDoBnQmEECJ8jfbw9IUnnaVR0h0',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': "'t1'!I20",
 'updatedRows': 1}

In [54]:
# Voting! Let's define the winner
worksheet.sheet1.update_cell(user_id, 10, "Vote_result_ID")

{'spreadsheetId': '1UEgjficfHAgOxfaZFDoBnQmEECJ8jfbw9IUnnaVR0h0',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': "'t1'!J20",
 'updatedRows': 1}